In [ ]:
import torch
a = torch.tensor([[1,2,3,4],[1,2,3,4]], dtype=float)
b = torch.ones((2, 2, 10, 20)) * 2
torch.var(a, dim=-1,keepdim=True, unbiased=False)

tensor([[1.2500],
        [1.2500]], dtype=torch.float64)

In [ ]:
a = torch.ones((2, 2, 10, 20))
b = torch.ones((2, 2, 10, 20)) * 2

theta = torch.arange(1, 21).broadcast_to((10, 20))

m = torch.arange(1, 11).reshape((10,1)).broadcast_to((10, 20))
mt = theta * m
mt, b * mt

(tensor([[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
           15,  16,  17,  18,  19,  20],
         [  2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,  26,  28,
           30,  32,  34,  36,  38,  40],
         [  3,   6,   9,  12,  15,  18,  21,  24,  27,  30,  33,  36,  39,  42,
           45,  48,  51,  54,  57,  60],
         [  4,   8,  12,  16,  20,  24,  28,  32,  36,  40,  44,  48,  52,  56,
           60,  64,  68,  72,  76,  80],
         [  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,  70,
           75,  80,  85,  90,  95, 100],
         [  6,  12,  18,  24,  30,  36,  42,  48,  54,  60,  66,  72,  78,  84,
           90,  96, 102, 108, 114, 120],
         [  7,  14,  21,  28,  35,  42,  49,  56,  63,  70,  77,  84,  91,  98,
          105, 112, 119, 126, 133, 140],
         [  8,  16,  24,  32,  40,  48,  56,  64,  72,  80,  88,  96, 104, 112,
          120, 128, 136, 144, 152, 160],
         [  9,  18,  27,  36,  4

In [ ]:
torch.stack((b * mt,a * mt), -1).flatten(-2)

tensor([[[[  2.,   1.,   4.,  ...,  19.,  40.,  20.],
          [  4.,   2.,   8.,  ...,  38.,  80.,  40.],
          [  6.,   3.,  12.,  ...,  57., 120.,  60.],
          ...,
          [ 16.,   8.,  32.,  ..., 152., 320., 160.],
          [ 18.,   9.,  36.,  ..., 171., 360., 180.],
          [ 20.,  10.,  40.,  ..., 190., 400., 200.]],

         [[  2.,   1.,   4.,  ...,  19.,  40.,  20.],
          [  4.,   2.,   8.,  ...,  38.,  80.,  40.],
          [  6.,   3.,  12.,  ...,  57., 120.,  60.],
          ...,
          [ 16.,   8.,  32.,  ..., 152., 320., 160.],
          [ 18.,   9.,  36.,  ..., 171., 360., 180.],
          [ 20.,  10.,  40.,  ..., 190., 400., 200.]]],


        [[[  2.,   1.,   4.,  ...,  19.,  40.,  20.],
          [  4.,   2.,   8.,  ...,  38.,  80.,  40.],
          [  6.,   3.,  12.,  ...,  57., 120.,  60.],
          ...,
          [ 16.,   8.,  32.,  ..., 152., 320., 160.],
          [ 18.,   9.,  36.,  ..., 171., 360., 180.],
          [ 20.,  10.,  40.,  .

In [ ]:
head_dim = 4
query = 2 * torch.ones([1, 2, 2, 4])
_, seqlen, _, _ = query.shape
device = query.device
theta = 1000


# Please refer to Lecture 5 slides in https://cmu-l3.github.io/anlp-fall2025/static_files/anlp-f2025-05-transformers.pdf
# and Section 3 in https://arxiv.org/abs/2104.09864.

# reshape xq and xk to match the complex representation
query_real, query_imag = query.float().reshape(query.shape[:-1] + (-1, 2)).unbind(-1)
# key_real, key_imag = key.float().reshape(key.shape[:-1] + (-1, 2)).unbind(-1)
# This separates each query/key vector into its odd and even indices (assuming *one-indexing*).
# query_real contains q_1, q_3, q_5, ... and query_imag contains q_2, q_4, q_6, ...
query_real = query_real.transpose(1,2)
query_imag = query_imag.transpose(1,2)
# key_real = key_real.transpose(1,2)
# key_imag = key_imag.transpose(1,2)

# First, compute the trigonometric values in the second and fourth columns in
# slide 49 (linked above).
print(1000 ** (-0.5))
dim = head_dim // 2
position = torch.arange(0, seqlen , device=device).reshape((seqlen, 1)).broadcast_to((seqlen, dim))
theta_rotate = torch.arange(0, dim , device=device).broadcast_to((seqlen, dim))
print("theta_init", theta_rotate)
theta_rotate = theta ** (-2.0 * theta_rotate / head_dim)
print("theta_1000", theta_rotate.dtype)
theta_rotate = position * theta_rotate
print("theta_m", theta_rotate)
sin_rotate = torch.sin(theta_rotate)
cos_rotate = torch.cos(theta_rotate)
print("cos", cos_rotate, sin_rotate)
print("half",query_real * cos_rotate)
print("half",query_imag * cos_rotate)
# Then, combine these trigonometric values with the tensors query_real, query_imag,
# key_real, and key_imag.
rotate_qleft = torch.stack((query_real * cos_rotate, query_imag * cos_rotate), dim=-1).flatten(-2)
print("left", rotate_qleft)
rotate_qright = torch.stack((-query_imag * sin_rotate, query_real * sin_rotate), dim=-1).flatten(-2)
print("right", rotate_qright)

query_out = (rotate_qleft + rotate_qright).transpose(1,2)

query_out

0.03162277660168379
theta_init tensor([[0, 1],
        [0, 1]])
theta_1000 torch.float32
theta_m tensor([[0.0000, 0.0000],
        [1.0000, 0.0316]])
cos tensor([[1.0000, 1.0000],
        [0.5403, 0.9995]]) tensor([[0.0000, 0.0000],
        [0.8415, 0.0316]])
half tensor([[[[2.0000, 2.0000],
          [1.0806, 1.9990]],

         [[2.0000, 2.0000],
          [1.0806, 1.9990]]]])
half tensor([[[[2.0000, 2.0000],
          [1.0806, 1.9990]],

         [[2.0000, 2.0000],
          [1.0806, 1.9990]]]])
left tensor([[[[2.0000, 2.0000, 2.0000, 2.0000],
          [1.0806, 1.0806, 1.9990, 1.9990]],

         [[2.0000, 2.0000, 2.0000, 2.0000],
          [1.0806, 1.0806, 1.9990, 1.9990]]]])
right tensor([[[[-0.0000,  0.0000, -0.0000,  0.0000],
          [-1.6829,  1.6829, -0.0632,  0.0632]],

         [[-0.0000,  0.0000, -0.0000,  0.0000],
          [-1.6829,  1.6829, -0.0632,  0.0632]]]])


tensor([[[[ 2.0000,  2.0000,  2.0000,  2.0000],
          [ 2.0000,  2.0000,  2.0000,  2.0000]],

         [[-0.6023,  2.7635,  1.9358,  2.0622],
          [-0.6023,  2.7635,  1.9358,  2.0622]]]])

In [ ]:
position, theta_rotate

(tensor([[1, 1],
         [2, 2]]),
 tensor([[0.0316, 0.0010],
         [0.0632, 0.0020]]))

tensor([[[[ 2.0000,  2.0000,  2.0000,  2.0000],
          [ 2.0000,  2.0000,  2.0000,  2.0000]],

         [[-0.6023,  2.7635,  1.9799,  2.0199],
          [-0.6023,  2.7635,  1.9799,  2.0199]]]])

In [ ]:
logits = torch.rand((2, 30, 3000))
logits = logits.softmax(dim=-1)
next = logits[:, -1, :].argmax(dim=-1, keepdim=True)
next

tensor([[1496],
        [1313]])

In [ ]:
t = logits[:,-1,:]
t = t / t.sum(dim=-1, keepdim=True)
t.shape

torch.Size([2, 3000])

In [ ]:
checkpoint_dict = torch.load("stories42M.pt", map_location='cpu')

In [ ]:
checkpoint_dict

{'model': OrderedDict([('tok_embeddings.weight',
               tensor([[-0.0550,  0.0157,  0.0512,  ...,  0.0158, -0.0063,  0.0015],
                       [-0.0058,  0.0129, -0.0105,  ...,  0.0077, -0.0085,  0.0352],
                       [-0.0550,  0.0157,  0.0512,  ...,  0.0159, -0.0063,  0.0015],
                       ...,
                       [-0.0550,  0.0157,  0.0512,  ...,  0.0158, -0.0063,  0.0015],
                       [-0.0550,  0.0157,  0.0512,  ...,  0.0159, -0.0063,  0.0015],
                       [-0.0550,  0.0157,  0.0512,  ...,  0.0158, -0.0063,  0.0015]])),
              ('layers.0.feed_forward.w1.weight',
               tensor([[-2.6168e-02,  3.1070e-02,  1.7426e-03,  ...,  1.3901e-02,
                         2.0365e-02, -5.8867e-02],
                       [-2.7580e-02,  1.0065e-02, -6.7414e-03,  ..., -6.3114e-02,
                        -2.5302e-02, -1.0540e-02],
                       [-1.5952e-02,  9.9807e-03, -3.0034e-03,  ...,  5.5196e-03,
            

In [ ]:
torch.log_softmax()